In [4]:
# Load in data and clean corpus

In [15]:
# Right. Let's get started.
# Determine initial allocation
import numpy as np
from collections import Counter

# Set aspects of the corpus: D is n.doc, N is n.words, k is n.topics
D = 2
k = 4
vocabulary = ['rabbit', 'horse', 'unicorn']
N = len(vocabulary)

# Set the hyperparameters
alpha = np.array([1] * k)
eta = [1] * N

# Initialise the document-specific mixing probabilities, D x k
theta = np.array([])
for doc in range(D):
    theta = np.append(theta, np.random.dirichlet(alpha))
theta = np.reshape(theta, (D, k))

# Initialise the topic allocations, D x k - that is, in document d topic k occured this many times
z = np.array([])
for doc in range(D):
    print doc
    z = np.append(z, np.random.multinomial(N, pvals = theta[doc], size = 1))
    
z = np.reshape(z, (D, k))

# Initialise the topic-specific term probabilities
beta = np.array([])
for topic in range(k):
    beta = np.append(beta, np.random.dirichlet(eta))
beta = np.reshape(beta, (k, N))

print beta
print w
print theta

0
1
[[  8.74735946e-02   4.61166924e-01   4.51359482e-01]
 [  6.55069787e-01   1.39849365e-01   2.05080848e-01]
 [  4.45305281e-01   1.93828518e-01   3.60866201e-01]
 [  1.23711430e-04   5.11918263e-01   4.87958025e-01]]
[['rabbit' 'rabbit' 'rabbit']
 ['rabbit' 'horse' 'rabbit']]
[[ 0.26905562  0.24257079  0.03151065  0.45686295]
 [ 0.20014146  0.27935246  0.07262112  0.44788496]]


In [16]:
# Start iterating
it = 1
total_results = []
while it < 1100:
    # Draw the word allocations
    w = np.array([])
    # Draw the word allocations
    for doc in range(D):
        for zk in range(len(z[doc])):
            w = np.append(w, np.random.choice(vocabulary, p = beta[zk], size = z[doc, zk]))
    w = np.reshape(w, (D, N))
    
    # Draw the new theta values
    alpha_new = z + alpha
    theta = np.array([])
    for doc in range(D):
        theta = np.append(theta, np.random.dirichlet(alpha_new[doc]))
    theta = np.reshape(theta, (D, k))
       
    # Draw the new beta values
    # Find the ms
    count_vocab = dict(zip(vocabulary, [0] * len(vocabulary))) 
    count_terms = Counter(w[0])
    for key in count_vocab:
        count_vocab[key] += count_terms[key]
    m = np.array(count_vocab.values())
    eta_new = m + eta
    # Find the betas
    beta = np.array([])
    for topic in range(k):
        beta = np.append(beta, np.random.dirichlet(eta_new))
    beta = np.reshape(beta, (k, N))
    
    # Draw the new zs
    z_new = np.array([])
    for doc in range(D):
        # Our first task is to find the document-specific mixing probabilities
        beta_theta2 = np.matrix(beta)
        beta_theta2 = np.transpose(beta_theta2)
        denominators = sum(beta_theta2)
        # First we find a N x k matrix of beta[k,n] *theta[d,k] 
        beta_theta = np.array([])
        denominator = [0] * N
        for row in range(len(beta)):
            be = beta[row] * theta[doc, row]
            denominator += be
            beta_theta = np.append(beta_theta, be)

        # We get this into a kxN matrix, because reasons
        beta_theta = np.reshape(beta_theta, (k, N))
        beta_theta = np.matrix(beta_theta)
        beta_theta = np.transpose(beta_theta)

        # And then we simulate the z values
        z_thisdoc = np.array([0] * k)
        for word in range(len(w[doc])):
            which_vocab = vocabulary.index(w[doc,word])
            this_mix = beta_theta[which_vocab] / denominator[which_vocab]
            z_thisdoc = z_thisdoc + np.random.multinomial(1, pvals = this_mix.getA1())
        z_new = np.append(z_new, z_thisdoc)
    z_new = np.reshape(z_new, (D, k))
    z = z_new.copy()
    
    # Every 50th iteration, we take a sample
    if it > 1000 & it % 50 == 0:
        results = []
        results.append(z)
        results.append(beta)
        results.append(theta)
        results.append(w)
        total_results.append(results)
    
    it += 1


/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:10: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
